In [1]:
import subprocess
import sys
import os
import re
import atexit
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
from IPython.display import clear_output

# Librería para evitar mensajes de warning al ejecutar código Python
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore')

# Librería para cronometrar tiempos de ejecución
#import time
#start_time = time.time()

## Carga de datos

In [2]:
df = pd.read_pickle('./sampledata2.pkl')

## Feature Engineering II

In [3]:
# Dataframe de features a las que aplicar medidas estadísticas móviles
moving_features = ['Date', 'DeviceID']
moving_features.extend([feature for feature in df.columns if 'Error' in feature and len(feature) == 13])
moving_features.extend([feature for feature in df.columns if 'Problem' in feature and len(feature) == 26])
moving_features.extend([feature for feature in df.columns if 'Fault' in feature and len(feature) == 29])
moving_features.extend([feature for feature in df.columns if 'Warning_Type' in feature and len(feature) == 26])
moving_features.extend([feature for feature in df.columns if 'AE' in feature])
df_moving = df[moving_features]

# Ventanas de longitug 3, 7, 15, 30 y 90 días
time_windows = ['3d', '7d', '15d', '30d', '90d']

In [4]:
for time_w in time_windows:
    dfs_devices_mean, dfs_devices_std, dfs_devices_max, dfs_devices_min = [], [], [], []
    dfs_devices_window = pd.DataFrame()
    
    # Media móvil
    for device in df_moving.DeviceID.unique():
        df_device = df_moving.loc[(df_moving.DeviceID == device)]
        df_device = df_device.drop('DeviceID', axis=1)
        df_device = df_device.set_index('Date')
        df_device = df_device.rolling(time_w).mean()
        dfs_devices_mean.append(df_device)
    
    print('Media móvil de', time_w, 'terminada.')
    dfs_devices_mean = pd.concat([df_device for df_device in dfs_devices_mean], ignore_index=True)
    dfs_devices_mean.columns = [feature + '_Roll_Mean_' + time_w for feature in dfs_devices_mean.columns]
    
    # Desviación estándar móvil
    for device in df_moving.DeviceID.unique():
        df_device = df_moving.loc[(df_moving.DeviceID == device)]
        df_device = df_device.drop('DeviceID', axis=1)
        df_device = df_device.set_index('Date')
        df_device = df_device.rolling(time_w).std()
        dfs_devices_std.append(df_device)
    
    print('Desv. estándar móvil de', time_w, 'terminada.')
    dfs_devices_std = pd.concat([df_device for df_device in dfs_devices_std], ignore_index=True)
    dfs_devices_std.columns = [feature + '_Roll_Std_' + time_w for feature in dfs_devices_std.columns]
    
    # Máximo móvil
    for device in df_moving.DeviceID.unique():
        df_device = df_moving.loc[(df_moving.DeviceID == device)]
        df_device = df_device.drop('DeviceID', axis=1)
        df_device = df_device.set_index('Date')
        df_device = df_device.rolling(time_w).max()
        dfs_devices_max.append(df_device)
    
    print('Máximo móvil de', time_w, 'terminado.')
    dfs_devices_max = pd.concat([df_device for df_device in dfs_devices_max], ignore_index=True)
    dfs_devices_max.columns = [feature + '_Roll_Max_' + time_w for feature in dfs_devices_max.columns]
    
    # Mínimo móvil
    for device in df_moving.DeviceID.unique():
        df_device = df_moving.loc[(df_moving.DeviceID == device)]
        df_device = df_device.drop('DeviceID', axis=1)
        df_device = df_device.set_index('Date')
        df_device = df_device.rolling(time_w).min()
        dfs_devices_min.append(df_device)
    
    print('Mínimo móvil de', time_w, 'terminado.')
    dfs_devices_min = pd.concat([df_device for df_device in dfs_devices_min], ignore_index=True)
    dfs_devices_min.columns = [feature + '_Roll_Min_' + time_w for feature in dfs_devices_min.columns]
    
    # Concatenación de las features obtenidas de las 4 medidas móviles y guardado de los datos
    dfs_devices_window = pd.concat([dfs_devices_mean, dfs_devices_std, dfs_devices_max, dfs_devices_min], axis=1)
    dfs_devices_window.to_pickle('./df_rolling_' + time_w + '.pkl')

Media móvil de 3d terminada.
Desv. estándar móvil de 3d terminada.
Máximo móvil de 3d terminado.
Mínimo móvil de 3d terminado.
Media móvil de 7d terminada.
Desv. estándar móvil de 7d terminada.
Máximo móvil de 7d terminado.
Mínimo móvil de 7d terminado.
Media móvil de 15d terminada.
Desv. estándar móvil de 15d terminada.
Máximo móvil de 15d terminado.
Mínimo móvil de 15d terminado.
Media móvil de 30d terminada.
Desv. estándar móvil de 30d terminada.
Máximo móvil de 30d terminado.
Mínimo móvil de 30d terminado.
Media móvil de 90d terminada.
Desv. estándar móvil de 90d terminada.
Máximo móvil de 90d terminado.
Mínimo móvil de 90d terminado.


## Guardado de datos

In [5]:
rolling_files = ['./df_rolling_' + time_w + '.pkl' for time_w in time_windows]

for r_file in rolling_files:
    df_rolling = pd.read_pickle(r_file)
    df_rolling = pd.concat([df, df_rolling], axis=1)
    df_rolling.to_pickle(r_file)
    print('El fichero', r_file, 'se ha guardado correctamente.')

El fichero ./df_rolling_3d.pkl se ha guardado correctamente.
El fichero ./df_rolling_7d.pkl se ha guardado correctamente.
El fichero ./df_rolling_15d.pkl se ha guardado correctamente.
El fichero ./df_rolling_30d.pkl se ha guardado correctamente.
El fichero ./df_rolling_90d.pkl se ha guardado correctamente.
